In [2]:
import numpy as np

In [3]:

class Ising:
    j = 1
    def __init__(self, L, T) -> None:
        self.rng = np.random.default_rng(seed=None)
        self.mag = self.CreateMagnet(L)
        self.L = L
        self.T = T

    def CreateMagnet(self, L):
        mag = self.rng.choice([1, -1], (L, L))
        
        return mag
    
    def takeStep(self):
        x = self.rng.integers(0, self.L)
        y = self.rng.integers(0, self.L)
        deltaE = self.checkEnergyChange(x, y)
        r = self.rng.random()
        if (r < np.exp(-deltaE/self.T)):
            self.mag[x, y] *= -1
        
    def checkEnergyChange(self, x, y):
        Ebefore = 0
        # if they are both 1 or -1, then their product will be 1, but we want E to be negative
        Ebefore += self.mag[x, y] * self.mag[x, (y+1) % self.L] * -self.j
        Ebefore += self.mag[x, y] * self.mag[x, (y-1) % self.L] * -self.j
        Ebefore += self.mag[x, y] * self.mag[(x+1) % self.L, y] * -self.j
        Ebefore += self.mag[x, y] * self.mag[(x-1) % self.L, y] * -self.j
        
        Eafter = 0
        Eafter += self.mag[x, y] * self.mag[x, (y+1) % self.L] * self.j
        Eafter += self.mag[x, y] * self.mag[x, (y-1) % self.L] * self.j
        Eafter += self.mag[x, y] * self.mag[(x+1) % self.L, y] * self.j
        Eafter += self.mag[x, y] * self.mag[(x-1) % self.L, y] * self.j
        
        return Eafter - Ebefore
    
    def sweep(self):
        for i in range(self.L * self.L):
            self.takeStep()

    def totalEnergy(self):
        E = 0
        
        pass

In [5]:
myMag = Ising(10, 1)
print(myMag.mag)
for i in range(10):
    myMag.sweep()
print(myMag.mag)

[[-1  1 -1 -1  1  1 -1  1  1 -1]
 [ 1  1 -1  1  1 -1  1  1  1  1]
 [ 1 -1 -1 -1 -1 -1 -1  1  1  1]
 [-1 -1  1 -1 -1 -1 -1 -1  1  1]
 [-1 -1  1  1 -1  1  1 -1 -1 -1]
 [-1 -1 -1 -1 -1  1  1 -1 -1 -1]
 [ 1 -1 -1  1  1  1 -1  1  1 -1]
 [-1 -1  1 -1  1  1  1  1  1  1]
 [-1  1 -1 -1 -1  1  1 -1  1 -1]
 [ 1 -1  1  1 -1 -1  1 -1 -1  1]]
[[-1 -1  1 -1 -1  1  1  1  1 -1]
 [-1 -1  1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1  1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  1  1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  1  1 -1 -1 -1 -1 -1 -1 -1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]
 [ 1  1  1  1  1  1  1  1  1  1]]
